# Inverting the Formula for the Ising Coupling Matrix 

## Notes on Network Performance

Trying to invert:

\begin{split}J_{i,j}&=\sum^N_{n=1}\Omega_{i,n}\Omega_{j,n}
  \sum^N_{m=1}&\frac{\eta_{i,m}\eta_{j,m}\omega_{m}}{\mu_{n}^2-\omega_{m}^2}
\end{split}

- where $\Omega$ is the unknown set of parameters and we construct out data set by sampling values of $\Omega$ from  $U(0,1)$ distribution.
- so far we have been using a simple feedforward network with $J_{i,j}$ as the inputs
- the sigmoid activation is used and we apply the tranformation $2x-1$ to the output of the last layer to obtain a prediction for $\Omega$

- We use cost function:
\begin{split}C_{x}&=\frac{1}{2}\sum_{j}&(y_{j}-(2a_{j}^{L}-1))^2
\end{split}
where L is the output layer and the sum is over all j in the outputlayer and $a^{l}_{i}$ denotes the activation of the $i$-th neuron in the $l$-th later.

Test Results:

- We have tested data corresponding to ionchains of size N=3 to 11. 
- The data sets we use are of size appears to have little impact although we use 100,000 data pairs split 60/20/20 into training set, validation set and test set.
- aside from N=10, we got that on an untrained network $\frac{Cost of test data}{Number of Outputs} \approx 0.26$ and once training had been completed $\frac{Cost of test data}{Number of Outputs} \approx 0.17$ so something was learned although the $J_{i,j}$ predictions resulting from the $\Omega$ predictions dont appear helpful 



Observations:

- most of the training is done on the first epoch
- $J_{i,j}$ depends only on the $i$-th and $j$-th rows of $\Omega$ and so it seems that some of the connections in the network may be unhelpful


In [30]:
import ionchain
import classes
import pickle
import numpy as np
import network
import network2
import matplotlib.pyplot as plt
import sklearn.preprocessing
% matplotlib inline

ionchain3 = ionchain.IonChain(3,[5,1])
ionchain4 = ionchain.IonChain(4,[5,1])
ionchain5 = ionchain.IonChain(5,[5,1])
ionchain6 = ionchain.IonChain(6,[5,1])
ionchain10 = ionchain.IonChain(10,[5,1])
ionchain7 = ionchain.IonChain(7,[5,1])
ionchain8 = ionchain.IonChain(8,[5,1])
ionchain9 = ionchain.IonChain(9,[5,1])
ionchain11 = ionchain.IonChain(11,[5,1])

## Functions to Model the Physical System
These functions, Lambe_Dicke J_ij1 and J_ij used to make the necessary calculations to model an instance of an ion chain.

In [31]:
def Lamb_Dicke(ic):
    """Computes Lamb-Dicke parameters for a trapped ion chain 
    
    Args:
        ic (Object) - an instance of a trapped ion chain
        
    Returns:
        An nxn matrix where n is the number of ions in the ion chain.
        The i,mth entry of the matrix corresponds to the Lambe-Dicke
        parameter eta[i,m] which sets the scale for the coupling between
        spin i and mode m.
    """
    deltak = 2*1.7699*10**7
    M = 2.8395 * 10**(-25)
    hbar = 1.0546e-34
    n = ic.n
    b_ij = ic.x_eigvecs
    omega_m = ic.x_freqs
    eta = np.empty([n,n])
    for m in range(n):
        for i in range(n):
            eta[i,m] = b_ij[i,m]*deltak*((hbar/(2*M*omega_m[m]))**0.5)
    return eta

In [32]:
def J_ij1(mu, Rabi_freq, ic):
    """Computes spin-spin coupling beteen atoms in a trapped ion
    chain with global beatnote detuning. 
    
    Args:
        mu(float) - global beatnote detuning parameter
        ic (Object) - an instance of a trapped ion chain
        Rabi_freq(An 1Xn vector of floats, where n is number of ions
        in the chain) 
            - containing where the ith entry is the single spin Rabi
            frequency of atom i
        
    Returns:
        An nxn matrix where n is the number of ions in the ion chain.
        The i,jth is the spin-spin coupling beteen atoms i and j in the
        ion chain.
    """
    n = ic.n
    eta = Lamb_Dicke(ic)
    omega_m = ic.x_freqs
    J = np.empty((n,n))
    for i in range(n):
        J[i,i] = 0
        for j in range(i+1,n):
            J[i,j] = Rabi_freq[i]*Rabi_freq[j]* np.sum(eta[i, :] * eta[j, :] * omega_m[:] /(np.full((1,n),mu)**2 - omega_m[:]**2))
    return J

In [33]:
def L_2(matrix):
    return np.linalg.norm(matrix)
def L_1(matrix):
    return np.sum(np.abs(matrix))

In [34]:
def J_ij(mus, Rabi_freq_matrix, ic, norm_function):
    n = ic.n
    eta = Lamb_Dicke(ic)
    omega_m = ic.x_freqs
    F = np.empty((n, n, len(mus)))
    for i in range(n):
        for j in range(n):
            for m in range(len(mus)):
                F[i, j, m] = np.sum(eta[i, :] * eta[j, :] * omega_m[:] /
                                    (mus[m]**2 - omega_m[:]**2))
    J = np.empty((n, n))
    for i in range(n):
        J[i, i] = 0
        for j in range(i + 1, n):
            J[i, j] = np.sum(Rabi_freq_matrix[i, :] * Rabi_freq_matrix[j, :] * F[i, j, :])
            J[j, i] = J[i, j]
            
    upper_triangular = J[np.triu_indices(n,1)]
    size_upper = int(n*(n-1)/2)
    norm = norm_function(upper_triangular)
    if norm != 0:
        normalized_J = J/norm
        normalized_upper_triangular = upper_triangular/norm
    else:
        normalized_J = J
        normalized_upper_triangular = upper_triangular
 
    J_dict = {'original' : J, 
              'upper triangular' : np.reshape(upper_triangular, (size_upper,1)),
              'norm' : norm,
              'normalized' : normalized_J,
              'normailzed upper triangular' : np.reshape(normalized_upper_triangular, (size_upper,1))}

    return J_dict

## Generating Synthetic Data
The functions mu_calculator, data_creator and create_data_file are all used to generate synthetic data. normalize is used to standardize the J_ij inputs.

In [35]:
def mu_calculator(ic):
    """Given an ionchain as input will calculate a beatnote detuning vector
    based on the transverse mode frequencies."""
    n = ic.n
    omega_m = ic.x_freqs
    mu = []
    for i in range(n-1):
        #alter the following line to change location of ith beatnote
        #detuning wrt to ith and i+1th transverse mode frequency
        mu_i = omega_m[i] + (omega_m[i+1]-omega_m[i])*0.1
        mu.append(mu_i)
    mu.append(omega_m[-1]*1.01)
    return mu 

In [36]:
 def data_creator(ic, data_size, norm_function):
    """Generates data for a particular ion chain ic.
    The data is returned in the form of a list of tuples.
    eg. [(x0,y0),(x1,y1),...] where (xi,yi) corresponds to an input output
    pair when it comes time to train the network. xi corresponds to the
    J_ijs and yi the Rabi_frquencies for the network.
    
    Note the xi and yis will be numpy vectors.
    
    Arguments:
    ic(Object from IonChain Class) - the ion chain from which to construct
    data from
    
    data_size(Int) - desired data size
    
    """
    n = ic.n
    omega_m = ic.x_freqs
    #Note: this function assumes that mu(beatnote detunings) are
    #predetermined and are calculated using the following line of code.
    #This function can be easily modified to change this.
    mu = mu_calculator(ic)
    #create dictionary to store information later
    d = {}
    #create list to store data later
    data = []
    for i in range (0, data_size):
        #alter this line of code to get frquencies from another
        #distribution
        Rabi = np.random.uniform(low=-1, high=1, size=(n,n))
        J_dict = J_ij(mu, Rabi, ic, norm_function)
        y = np.reshape(Rabi,(n**2,1))
        #alter this line of code so that input data is different
        data.append((J_dict['normailzed upper triangular'],y))
    d['data'] = data
    d['ionchain'] = ic
    d['data_size'] = data_size
    return d

In [ ]:
def create_data_file(filename, ic, data_size, norm_function):
    """Creates a pickled file containing synthetic test data.
    Arguments:
    filename(string)-desired filename
    ic(Object from IonChain Class)-the ion chain from which to construct
    data from
    data_size(Int)-desired data size
    triangular(Bool) - if only want the input data to be entries of J_ij
    above the diagonal"""
    d = data_creator(ic, data_size, norm_function)
    data = d['data']
    d['training_set'] = data[:int(len(data)*0.6)]
    d['validation_set'] = data[int(len(data)*0.6) :int(len(data)*0.8)]
    d['test_set'] = data[int(len(data)*0.8):]
    f = open(filename, 'wb')
    pickle.dump(d, f)
    f.close()
    
#create_data_file('data_L1.pickle', ionchain3, 1000000, L_1)
#create_data_file('data_ic4_L1.pickle', ionchain4, 100000, L_1)
create_data_file('data_ic15_L1.pickle', ionchain15, 10000, L_1)
d = pickle.load(open('data_ic15_L1.pickle', "rb"))
net1 = network2.Network([105,500,225])
net1.total_cost_no_reg(d['training_set'])

## Training and Validation
The following code is used to define a create a neural network and its architecture, using a modified version of Micheal Nielsons network2.py file, then train it. After running this code the output will currently be the cost for the training and test data at the end of each epoch stored in an array. Along with a plot for visualization.


In [ ]:
def train(filename,architecture,epochs,mini_batch,eta,lmbda=0,save=False,
         save_name=None):
    d = pickle.load(open(filename, "rb"))
    ic = d['ionchain']
    training_set = d['training_set']
    validation_set = d['validation_set']
    test_set = d['test_set']
    net = network2.Network(architecture)
    net.SGD(training_set, epochs, mini_batch, eta ,lmbda,
        validation_set,eta_update=True)
    if save:
        if save_name:
            d['sizes'] = architecture
            d['weights'] = net.weights
            d['biases'] = net.biases
            d['cost'] = net.cost
            f = open(save_name, 'wb')
            pickle.dump(d, f)
            f.close()    
        else:
            print ('Must specify filename, to argument save_name to save network to.')
    return net.total_cost_no_reg(test_set)  

In [ ]:
train('data_ic15_L1.pickle'',[105,500,225],20,10,0.01,save=True,
     save_name='data_L1_ic15_network_1.pickle')


## Testing
The following code can be modified and used for testing.

In [ ]:
def comparison(J,filename):
    """Given an Ising coupling matrix J and a trained neural network """
    d = pickle.load(open(filename, "rb"))
    ic = d['ionchain']
    n = ic.n
    input_size = int(n*(n-1)/2)
    net = network2.Network(d['sizes'],d['cost'])
    net.weights = d['weights']
    net.biases = d['biases']
    J_upper = J[np.triu_indices(n,1)]
    norm = np.linalg.norm(J_upper)
    if norm != 0:
        J_normalized = J/norm
        J_upper_normalized = np.reshape(J_upper/norm, (input_size,1))
    else:
        J_normalized = J
        J_upper_normalized = np.reshape(J_upper, (input_size,1)) 
    #prints the normalized J so that it can be compared to the prediction
    print (J_normalized)
    nn_prediction = 2*net.feedforward(J_upper_normalized)-1
    mu = mu_calculator(ic)
    Rabi_approx = np.reshape(nn_prediction,(n,n))
    J_approx = J_ij(mu, Rabi_approx, ic, L_1)
    return J_approx['normalized']

In [ ]:
#comparison(np.array([[0,1,0],[1,0,1],[0,1,0]]),'data_L1_network_1.pickle')
#comparison(np.array([[0,1,0,0,0],[1,0,1,0,0],[0,1,0,1,0],[0,0,1,0,1],[0,0,0,1,0]]),'data_L1_ic5_network_1.pickle')